# Testing the classifier

In [10]:
## import necessary libraries
import pandas as pd

from helper_functions import tfidf_vector
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [11]:
## read in data
instances = pd.read_csv('data/ships_extended.csv', index_col=0)
X_train, X_test, Y_train, Y_test = train_test_split(instances.drop(['epithet_gr', 'epithet_en', 'clause'], axis=1), instances['epithet_en'], test_size=0.2, random_state=20016)

## Train model

In [12]:
## featurise
X_train, X_test = tfidf_vector(X_train['trigrams'], X_test['trigrams'])

In [13]:
## model
tree = DecisionTreeClassifier(max_depth=1)
tree = tree.fit(X_train, Y_train)

## Predict epithet

In [14]:
## make feature dataframe
test = Y_test
test = test.to_frame()

## add predictions to dataframe
test['prediction'] = tree.predict(X_test)
test.sort_index()

,epithet_en,prediction
8,none,none
9,hollow,none
12,none,none
14,swift,none
21,swift,none
32,swift,none
36,none,none
47,none,none
48,hollow,none
54,black,none


## Analyse model

In [15]:
## overall score
tree.score(X_test, Y_test)

0.5535714285714286

In [16]:
## create accuracy dicy
epithets = {}
for epithet in test['epithet_en'].unique():
    epithets[epithet] = {'tp':0, 'fp':0, 'fn':0, 'tn':0}

for index, row in test.iterrows():
    if row['epithet_en'] == row['prediction']:
        for epithet in epithets.keys():
            if epithet==row['epithet_en']:
                epithets[epithet]['tp']+=1
            else:
                epithets[epithet]['tn']+=1
    else:
        for epithet in epithets.keys():
            if epithet==row['epithet_en']:
                epithets[epithet]['fn']+=1
            elif epithet==row['prediction']:
                epithets[epithet]['fp']+=1
            else:
                epithets[epithet]['tn']+=1

In [17]:
## accuracy
accuracy = []
fp = []
fn = []
for epithet in epithets.keys():
    acc = (epithets[epithet]['tp']+epithets[epithet]['tn'])/len(Y_test)
    accuracy.append([epithet, acc, epithets[epithet]['fp'], epithets[epithet]['fn']])

acc = pd.DataFrame(accuracy)
acc.columns = ['epithet_en', 'accuracy', 'false_positives', 'false_negatives']
acc.sort_values(by='accuracy', ascending=False)

,epithet_en,accuracy,false_positives,false_negatives
3,well-made,0.982143,0,1
7,well-benched,0.982143,0,1
9,curved,0.982143,0,1
10,with oars,0.982143,0,1
5,dark-prowed,0.964286,0,2
6,well-balanced,0.964286,0,2
8,hollowed,0.964286,0,2
1,black,0.928571,0,4
4,hollow,0.928571,0,4
2,swift,0.875000,0,7


In [18]:
## average precision and recall
num = 0
prec_denom = 0
recall_denom = 0

for epithet in epithets.keys():
    num += epithets[epithet]['tp']

    prec_denom += epithets[epithet]['tp']
    prec_denom += epithets[epithet]['fp']

    recall_denom += epithets[epithet]['tp']
    recall_denom += epithets[epithet]['fn']

print('Precision:', num/prec_denom)
print('Recall:', num/recall_denom)

Precision: 0.5535714285714286
Recall: 0.5535714285714286
